# Gather Fitbit Data from JSON

Fitbit data arrives broken up into a series of JSON files, each containing a different range of dates and the corresponding values on a type of metric (exercise, heart rate, sleep, etc.). 

To speed construction of the data into a single dataframe, the following code pulls similarly labeled files (such as all of those with 'steps' in the filename), normalizes the JSON into a 'flat' dataframe and each of the desired values are appended to lists. When each file has been read, the lists are zipped into a single pandas dataframe.

Additional processing steps, such as converting time zones to local (from UTC), ensuring the dates are in a consistent format (for matching the dataframes by date), and converting some metrics to more readily interpretable forms (milliseconds to seconds).

In [1]:
import pandas as pd
import numpy as np

import json
from pandas.io.json import json_normalize

import datetime
from pytz import timezone

from os import listdir

## Daily Steps Taken

In [2]:
dir_path = 'health/Data/Raw Data/'
matching_phrase = 'steps-'

date = []
value = []

for file in listdir(dir_path):
    if matching_phrase in file:
        with open(dir_path + file) as f:
            data = json.load(f)
            df = pd.json_normalize(data)
            df.columns = ['date', 'value']

            for i in df['date']:
                date.append(i)

            for i in df['value']:
                value.append(i)

# zip lists into single dataframe
df = pd.DataFrame(list(zip(date, value)), 
   columns =['date', 'value'])

# convert time data to local timezone (CST)
df['date'] = pd.to_datetime(df['date']).dt.tz_localize('America/Chicago', 
                                                       ambiguous = 'NaT',
                                                       nonexistent='shift_forward')

df['date'] = pd.to_datetime(df['date'])

# remove time from date for grouping
df['day'] = pd.to_datetime(df['date']).dt.date

# convert value to numeric
df['value'] = pd.to_numeric(df['value'], errors = 'coerce')

Steps data from Fitbit come in minute intervals. For analysis, only the daily level is of interest so the dataframe to be returned needs these values aggregated by date.

In [3]:
steps = df.copy()

steps = steps.groupby(["day"]).agg(steps=("value","sum")).reset_index()

steps = steps.rename(columns={'day': 'date'})

steps

,date,steps
0,2018-02-09,10
1,2018-02-10,7964
2,2018-02-11,7582
3,2018-02-12,12535
4,2018-02-13,18006
...,...,...
625,2019-11-04,0
626,2019-11-05,32
627,2019-11-06,0
628,2019-11-07,0


## Daily Average Resting Heart Rate

In [4]:
matching_phrase = 'heart'

date = []
value = []

for file in listdir(dir_path):
    if matching_phrase in file:
        with open(dir_path + file) as f:
            data = json.load(f)
            df = pd.json_normalize(data)
            # this json has four columns and the JSON is formatted differently than the steps data
            df.columns = ['date_utc', 'date', 'rest_avg_hr', 'error']

            for i in df['date']:
                date.append(i)

            for i in df['rest_avg_hr']:
                value.append(i)

df = pd.DataFrame(list(zip(date, value)), 
   columns =['date', 'rest_avg_hr'])

# gather only dates with data
heart_rate = df.copy()

heart_rate = heart_rate[heart_rate['date'].notna()]

heart_rate['date'] = pd.to_datetime(heart_rate['date']).dt.date

In [5]:
heart_rate

,date,rest_avg_hr
1,2018-02-09,58.000000
2,2018-02-10,59.029068
3,2018-02-11,62.038370
4,2018-02-12,62.245052
5,2018-02-13,60.857355
...,...,...
622,2019-10-23,55.536179
623,2019-10-24,55.587471
624,2019-10-25,56.973512
625,2019-10-26,57.959826


## Daily Sleep Data

In [6]:
matching_phrase = 'sleep'

date = []
start_time = []
end_time = []
duration = []
asleep_min = []
awake_min = []
rem_min = []
rem_30_day_avg = []

for file in listdir(dir_path):
    if matching_phrase in file:
        with open(dir_path + file) as f:
            data = json.load(f)
            df = pd.json_normalize(data)
            # this json has several levels of nested data, select only those desired for analysis later
            df = df[['dateOfSleep', 'startTime', 'endTime','duration','minutesAsleep','minutesAwake',
               'levels.summary.rem.minutes','levels.summary.rem.thirtyDayAvgMinutes']]
            
            for i in df['dateOfSleep']:
                date.append(i)
                
            for i in df['startTime']:
                start_time.append(i)
                
            for i in df['endTime']:
                end_time.append(i)
                
            for i in df['duration']:
                duration.append(i)
                
            for i in df['minutesAsleep']:
                asleep_min.append(i)
                
            for i in df['minutesAwake']:
                awake_min.append(i)

            for i in df['levels.summary.rem.minutes']:
                rem_min.append(i)
                
            for i in df['levels.summary.rem.thirtyDayAvgMinutes']:
                rem_30_day_avg.append(i)         
                
sleep = pd.DataFrame(list(zip(date, start_time, end_time, duration, asleep_min, awake_min, rem_min, rem_30_day_avg)), 
   columns =['date', 'sleep_start_time', 'sleep_end_time', 'sleep_duration', 'asleep_min', 'awake_min', 'rem_min', 'rem_30_day_avg'])

# duration is in ms (divide by 60000 to obtain minutes)
sleep['sleep_duration'] = sleep['sleep_duration'] / 60000

# ensure date formatted similarly
sleep['date'] = pd.to_datetime(sleep['date']).dt.date

In [7]:
sleep

,date,sleep_start_time,sleep_end_time,sleep_duration,asleep_min,awake_min,rem_min,rem_30_day_avg
0,2018-03-08,2018-03-07T21:54:00.000,2018-03-08T07:00:30.000,546.0,502,44,83.0,109.0
1,2018-03-06,2018-03-05T21:36:00.000,2018-03-06T06:21:00.000,525.0,482,43,138.0,107.0
2,2018-03-03,2018-03-02T22:47:30.000,2018-03-03T07:50:30.000,543.0,481,62,142.0,103.0
3,2018-02-28,2018-02-27T21:52:00.000,2018-02-28T06:21:30.000,509.0,474,35,102.0,103.0
4,2018-02-26,2018-02-25T21:57:00.000,2018-02-26T06:25:30.000,508.0,444,64,57.0,109.0
...,...,...,...,...,...,...,...,...
498,2019-10-04,2019-10-03T20:47:00.000,2019-10-04T06:17:00.000,570.0,460,110,96.0,92.0
499,2019-10-03,2019-10-03T01:39:00.000,2019-10-03T06:28:00.000,289.0,255,34,69.0,103.0
500,2019-10-02,2019-10-02T22:00:30.000,2019-10-02T23:47:00.000,106.0,102,4,NaN,NaN
501,2019-10-02,2019-10-01T22:27:00.000,2019-10-02T06:34:30.000,487.0,433,54,112.0,94.0


## Combined Data

In [8]:
# make a list of dataframes to join, set date as index (required for .join())
dfs = [steps, heart_rate, sleep]
dfs = [df.set_index('date') for df in dfs]
fitbit_df = pd.DataFrame().join(dfs, how="outer")
fitbit_df

,steps,rest_avg_hr,sleep_start_time,sleep_end_time,sleep_duration,asleep_min,awake_min,rem_min,rem_30_day_avg
date,,,,,,,,,
2018-02-09,10,58.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-02-10,7964,59.029068,2018-02-09T22:14:30.000,2018-02-10T09:12:00.000,657.0,586.0,71.0,131.0,0.0
2018-02-11,7582,62.038370,2018-02-10T22:19:00.000,2018-02-11T09:19:30.000,660.0,579.0,81.0,133.0,131.0
2018-02-12,12535,62.245052,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-02-13,18006,60.857355,2018-02-12T22:34:30.000,2018-02-13T06:21:30.000,467.0,406.0,61.0,69.0,132.0
...,...,...,...,...,...,...,...,...,...
2019-11-04,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-11-05,32,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-11-06,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
#fitbit_df.to_csv('fitbit_clean.csv', index=True)

## Running Data

In [10]:
matching_phrase = 'exercise'

start_time = []
steps = []
distance = []
calories = []
elevationGain = []
speed = []
pace = []
duration = []
activeDuration = []
averageHeartRate = []
logType = []
activityName = []


for file in listdir(dir_path):
    if matching_phrase in file:
        with open(dir_path + file) as f:
            data = json.load(f)
            df = pd.json_normalize(data)
            
            for i in df['startTime']:
                start_time.append(i)
                
            for i in df['steps']:
                steps.append(i)
                
            for i in df['distance']:
                distance.append(i)
                
            for i in df['calories']:
                calories.append(i)
                
            for i in df['elevationGain']:
                elevationGain.append(i)
                
            for i in df['speed']:
                speed.append(i)
                
            for i in df['pace']:
                pace.append(i)
                
            for i in df['duration']:
                duration.append(i)
                
            for i in df['activeDuration']:
                activeDuration.append(i)
                
            for i in df['averageHeartRate']:
                averageHeartRate.append(i)
                
            for i in df['logType']:
                logType.append(i)
                
            for i in df['activityName']:
                activityName.append(i)
            
df = pd.DataFrame(list(zip(start_time, steps, distance, calories, elevationGain, speed, 
                               pace, duration, activeDuration, averageHeartRate, logType, activityName)),
   columns =['run_start_time', 'steps_exercise', 'fitbit_distance', 'calories', 'fitbit_elev_gain', 'avg_speed',
            'pace', 'duration', 'active_duration', 'avg_hr', 'log_type', 'activity_type'])

In [11]:
# limit activity data to tracked runs
run_df = df.copy()

run_df = run_df[(run_df['activity_type'] == 'Run') & (run_df['log_type'] == 'tracker')]

# duration is in ms (divide by 60000 to obtain minutes)
run_df['duration'] = run_df['duration'] / 60000
run_df['active_duration'] = run_df['active_duration'] / 60000

# ensure date formatted similarly and pull out starting time of run
run_df['date'] = run_df.run_start_time.apply(pd.to_datetime)
run_df['run_start_time'] = [d.time() for d in run_df['date']]
run_df['date'] = [d.date() for d in run_df['date']]
run_df = run_df.drop(['log_type','activity_type'], axis = 1)
run_df

,run_start_time,steps_exercise,fitbit_distance,calories,fitbit_elev_gain,avg_speed,pace,duration,active_duration,avg_hr,date
55,15:02:09,3016.0,2.127550,265,87.998688,7.336379,490.705271,17.400000,17.400000,169.0,2018-03-05
58,16:17:09,3024.0,2.134329,262,95.000000,7.283018,494.300551,17.583333,17.583333,168.0,2018-03-06
62,22:21:07,3055.0,2.161421,259,97.998688,7.361510,489.030133,17.616667,17.616667,168.0,2018-03-07
78,15:00:58,3044.0,2.149777,296,101.000656,7.146073,503.773182,18.050000,18.050000,172.0,2018-03-19
85,21:10:42,3272.0,2.306430,294,106.000656,7.232707,497.738930,19.133333,19.133333,171.0,2018-03-21
...,...,...,...,...,...,...,...,...,...,...,...
1090,13:37:18,2978.0,2.171382,233,73.999344,7.686308,468.365308,28.966667,16.950000,151.0,2019-08-29
1093,12:31:54,2976.0,2.144264,210,78.999344,7.635361,471.490451,25.800000,16.850000,146.0,2019-08-30
1102,12:54:32,2977.0,2.182485,231,102.001312,7.680299,468.731744,24.483333,17.050000,151.0,2019-09-03
1105,13:12:31,2965.0,2.169349,227,81.000656,7.709434,466.960365,23.633333,16.883333,148.0,2019-09-04


In [ ]:
#run_df.to_csv('running_fitbit.csv', index=False)